In [30]:
import cv2,os,shutil,requests
import numpy as np
import time
import pickle
import winsound

def Scale(img,scale_x= 0.3 ,scale_y= 0.3 ):
    img_shape = img.shape
    resize_scale_x = 1 + scale_x
    resize_scale_y = 1 + scale_y
    img =  cv2.resize(img, None, fx = resize_scale_x, fy = resize_scale_y)   
    canvas = np.zeros(img_shape, dtype = np.uint8)
    y_lim = int(min(resize_scale_y,1)*img_shape[0])
    x_lim = int(min(resize_scale_x,1)*img_shape[1])       
    canvas[:y_lim,:x_lim,:] =  img[:y_lim,:x_lim,:]
    return canvas  

def rotate(img,i):
    return cv2.flip(img, i)

def change_brightness(img, value):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    if value > 0:
        lim = 255 - value
        v[v > lim] = 255
        v[v <= lim] += value
        v[v == value] = 0
    else:
        lim = -value
        v[v < lim] = 0
        v[v >= lim] = v[v >= lim] + value 

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

def augmentation(source_path,target_path,target_size,values,flag=True):
    high = values['high']
    low =  values['low']
    
    target_size -= 1
    images = os.listdir(source_path)

    for img in images:
        
        image = cv2.imread(os.path.join(source_path,img))

        if flag:
            tempfolder = 'tempfolder'
            if not os.path.exists(tempfolder):
                os.mkdir(tempfolder)
            remove_from_folder(tempfolder)
            image_path = os.path.join(source_path,img)
            shutil.copy(image_path,tempfolder)

            for j in [high,low]:
                cv2.imwrite(os.path.join(tempfolder,os.path.splitext(os.path.basename(img))[0])+'_brightness_changed_{}.jpeg'.format(j),change_brightness(image, value=j), [cv2.IMWRITE_JPEG_QUALITY, 100])
            augmentation(tempfolder,target_path,target_size,values,flag=False)
        else:
        
            if  len(os.listdir(target_path)) <= target_size:
                cv2.imwrite(os.path.join(target_path,os.path.splitext(os.path.basename(img))[0])+'_scaled_{}.jpeg'.format(0.25),Scale(image,scale_x= 0.25,scale_y=0.25), [cv2.IMWRITE_JPEG_QUALITY, 100])
            else:break

            if  len(os.listdir(target_path)) <= target_size:
                cv2.imwrite(os.path.join(target_path,os.path.splitext(os.path.basename(img))[0])+'_rotated_cw.jpeg',rotate(image,cv2.ROTATE_90_CLOCKWISE), [cv2.IMWRITE_JPEG_QUALITY, 100])
            else:break

            if  len(os.listdir(target_path)) <= target_size:
                cv2.imwrite(os.path.join(target_path,os.path.splitext(os.path.basename(img))[0])+'_rotated_acw.jpeg',rotate(image,cv2.ROTATE_90_COUNTERCLOCKWISE), [cv2.IMWRITE_JPEG_QUALITY, 100])
            else:break

def do_augmetation(source_path,target_path,target_size,default,do_aug_again=False):
    values = {}
    values['high'] = default
    values['low'] = -default
    
    if not os.path.exists(target_path): # make the target folder
        os.mkdir(target_path)
        
    if not os.path.exists('brightness_values.txt'):
        print('No brightness_values.txt detected! Creating new...')
        with open("brightness_values.txt","wb") as f:
            brightness_values = {}
            pickle.dump(brightness_values, f)

    with open("brightness_values.txt", "rb") as f:
        try:
            brightness_values = pickle.load(f)
            print('brightness_values found and loaded!')
        except:
            print('Loading values for brightness_values.txt failed. Adding dictionary.')
            with open("brightness_values.txt", "wb") as f:
                brightness_values = {}
                pickle.dump(brightness_values, f)
        
    for path in os.listdir(source_path): # find the different folders in cwd
        
        target_path_new = os.path.join(target_path,path)
        source_path_new = os.path.join(source_path,path)
                
        if path not in brightness_values.keys():
            if not os.path.exists(target_path_new): # make the storage folder with same name
                os.mkdir(target_path_new)
                print(path,'CREATED!')
            check_brightness_values(source_path_new,target_path_new,values,desired_images = 10)
            print("values:",values,'from check_brightness_values')
            brightness_values['{}'.format(path)] = values
            with open("brightness_values.txt", "wb") as f:
                pickle.dump(brightness_values, f)
            augmentation(source_path_new,target_path_new,target_size,values) # do aug. and store there
            print('{} done'.format(path))

        else:
            if do_aug_again:
                print('removing prexisting files from {}'.format(path))
                if os.path.exists(target_path_new):
                    remove_from_folder(target_path_new)
                    print('removed')
                    
            if not os.path.exists(target_path_new): # make the storage folder with same name
                os.mkdir(target_path_new)
                print(path,'CREATED!')
                
            bv = brightness_values['{}'.format(path)]
            print('brightness values found!',bv)
            augmentation(source_path_new,target_path_new,target_size,bv)
            print('{} done'.format(path))

def remove_from_folder(folder_path):
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path,file)
        os.remove("{}".format(file_path))
        
def make_noise(i):
    duration = 200  # milliseconds
    freq = 2240     # Hz
    for i in range(i):
            winsound.Beep(freq, duration)

def check_brightness_values(source_path,target_path,values,desired_images = 10):
    high = values['high']
    low =  values['low']
    
    remove_from_folder(target_path)
    for i,img in enumerate(os.listdir(source_path)):
        if i <= desired_images - 1:
            image_path = os.path.join(source_path,img)
            shutil.copy(image_path,target_path)
    for i,img in enumerate(os.listdir(source_path)):
        if i <= desired_images - 1:
            for j in [high,low]:
                image = cv2.imread(os.path.join(source_path,img))
                cv2.imwrite(os.path.join(target_path,os.path.splitext(os.path.basename(img))[0])+'_brightness_changed_{}.jpeg'.format(j),change_brightness(image, value=j), [cv2.IMWRITE_JPEG_QUALITY, 100])
    make_noise(5)
    answer = input('Editted {}.Are you satisfied with the result?'.format(target_path))
    if answer == 'y':
        return 
    
    else:
        h,l = answer.split(',')
        if not int(l) <0:
            l=-1*int(l)
        values['high'],values['low'] = int(h),int(l)      
        check_brightness_values(source_path,target_path,values,desired_images)
            

def rename(source_path):
    for path in os.listdir(source_path):
        folder_path = os.listdir(os.path.join(source_path,path))
        
        for i,image in enumerate(folder_path): # creating temporary name to avoid fileexistserror
            old_name = source_path+'/'+path+'/'+image
            temp_name = source_path+'/'+path+'/'+str(i)+'.jpeg'
            os.rename(old_name,temp_name)
            
        for i,image in enumerate(folder_path): # folder_path - list of different images 
            name = str(os.path.splitext(os.path.basename(path))[0]) +" "+str(i) + '.jpeg'
#             try:
            temp_name = source_path+'/'+path+'/'+str(i)+'.jpeg'
            new_name = source_path+'/'+path+'/'+name
            os.rename(temp_name,new_name)
#             except FileExistsError:
#                 print('error')
#                 continue
    print(source_path,'done')
    
# remove underscores in the names of subfolders in a source_path    
def remove_s(source_path):
    for folder in os.listdir(source_path):
        folder_new = folder.replace("_"," ")
        os.rename(source_path+'/'+folder,source_path+'/'+folder_new)
    print("_s in",source_path,"removed")

In [24]:
remove_s('PlantVillage-Dataset/raw/segmented')

_s in PlantVillage-Dataset/raw/segmented removed


In [2]:
# happy_dog = cv2.imread('Happy_dog_augmentation_test_folder/leaf.jpeg')

# cv2.imshow("happy dog",happy_dog)

# scaled_happy_dog = Scale(happy_dog)
# rotated_clockwise = rotate(happy_dog,cv2.ROTATE_90_CLOCKWISE)
# rotated_c_clockwise = rotate(happy_dog,cv2.ROTATE_90_COUNTERCLOCKWISE)
# brightness_boosted = change_brightness(happy_dog, value=30)
# brightness_dimmed = change_brightness(happy_dog, value=-30)

# # try:
# # cv2.imshow("scaled",scaled_happy_dog)
# # cv2.imwrite("Happy_dog_augmentation_test_folder/scaled.jpg",scaled_happy_dog, [cv2.IMWRITE_JPEG_QUALITY, 100])

# # cv2.imshow("rotated clockwise",rotated_clockwise)
# # cv2.imwrite("Happy_dog_augmentation_test_folder/rotated clockwise.jpg",rotated_clockwise, [cv2.IMWRITE_JPEG_QUALITY, 100])

# # cv2.imshow("rotated counter clockwise",rotated_c_clockwise)
# # cv2.imwrite("Happy_dog_augmentation_test_folder/rotated counter clockwise.jpg",rotated_c_clockwise, [cv2.IMWRITE_JPEG_QUALITY, 100])

# cv2.imshow("brightness boosted",brightness_boosted)
# cv2.imwrite("Happy_dog_augmentation_test_folder/brightness boosted.jpg",brightness_boosted, [cv2.IMWRITE_JPEG_QUALITY, 100])

# cv2.imshow("brightness dimmed",brightness_dimmed)
# cv2.imwrite("Happy_dog_augmentation_test_folder/brightness boosted.jpg",brightness_dimmed, [cv2.IMWRITE_JPEG_QUALITY, 100])

# cv2.waitKey(0)
# cv2.destroyAllWindows()
# # except:
# # #     time.sleep(20)
# #     print('something went wrong')
# #     cv2.destroyAllWindows()
    
# # time.sleep(20)
# # print('time out')
# # cv2.destroyAllWindows()

In [3]:
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [4]:
# cv2.destroyAllWindows()

In [31]:
target_size = 7000
default = 70

do_augmetation('PlantVillage-Dataset/raw/segmented','New segmented folder',target_size,default,do_aug_again=False)

brightness_values found and loaded!
Apple   Apple scab CREATED!
brightness values found! {'high': 50, 'low': -50}
Apple   Apple scab done
Apple   Black rot CREATED!
brightness values found! {'high': 40, 'low': -60}
Apple   Black rot done
Apple   Cedar apple rust CREATED!
brightness values found! {'high': 40, 'low': -60}
Apple   Cedar apple rust done
Apple   healthy CREATED!
brightness values found! {'high': 40, 'low': -60}
Apple   healthy done
Cherry (including sour)   healthy CREATED!
brightness values found! {'high': 40, 'low': -60}
Cherry (including sour)   healthy done
Cherry (including sour)   Powdery mildew CREATED!
brightness values found! {'high': 40, 'low': -60}
Cherry (including sour)   Powdery mildew done
Corn (maize)   Cercospora leaf spot Gray leaf spot CREATED!
brightness values found! {'high': 50, 'low': -60}
Corn (maize)   Cercospora leaf spot Gray leaf spot done
Corn (maize)   Common rust CREATED!
brightness values found! {'high': 50, 'low': -50}
Corn (maize)   Common 

In [94]:
# rename('New-Dataset/waste folder')
# augmentation('Happy_dog_augmentation_test_folder/','subfolder',target_size=5)
target_size = 7000
default = 70
# do_augmetation('New-Dataset-Segmented','New-Dataset-Segmented-Augmented',target_size,default)
do_augmetation('New-Dataset-Segmented/','New-Dataset-Segmented-Augmented//',target_size,default,do_aug_again=False)
# augmentation(source_path='PlantVillage-Dataset/raw/segmented/Orange___Haunglongbing_(Citrus_greening)',target_path='New-Dataset/Orange___Haunglongbing_(Citrus_greening)')

removing prexisting files from Apple   Apple scab
removed
brightness values found! {'high': 50, 'low': -50}
Apple   Apple scab done
removing prexisting files from Apple   Black rot
removed
brightness values found! {'high': 40, 'low': -60}
Apple   Black rot done
removing prexisting files from Apple   Cedar apple rust
removed
brightness values found! {'high': 40, 'low': -60}
Apple   Cedar apple rust done
removing prexisting files from Apple   healthy
removed
brightness values found! {'high': 40, 'low': -60}
Apple   healthy done
removing prexisting files from Cherry (including sour)   healthy
removed
brightness values found! {'high': 40, 'low': -60}
Cherry (including sour)   healthy done
removing prexisting files from Cherry (including sour)   Powdery mildew
removed
brightness values found! {'high': 40, 'low': -60}
Cherry (including sour)   Powdery mildew done
removing prexisting files from Corn (maize)   Cercospora leaf spot Gray leaf spot
removed
brightness values found! {'high': 50, 'l

In [119]:
shapes = []
parent = 'New-Dataset-Segmented/'
for folder in os.listdir(parent):
    shapes.append(len(os.listdir(parent+'/'+folder)))

In [91]:
source_path = 'New-Dataset-Segmented-Augmented'
# remove_s(source_path)
rename(source_path)

New-Dataset-Segmented-Augmented done


In [19]:
output

10

In [18]:
output = test()

False


In [17]:
output = 0